<a href="https://colab.research.google.com/github/arjun-2131/cow-breed/blob/master/cow_breed_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
from keras.models import Sequential,Model
from keras.layers import Convolution2D,MaxPooling2D,GlobalAveragePooling2D
from keras.layers import Activation,Dropout,Flatten,Dense,BatchNormalization
from keras import backend as K
from keras.preprocessing import image
from keras import applications
from keras.callbacks import ModelCheckpoint,EarlyStopping

In [0]:
from keras.preprocessing.image import ImageDataGenerator,array_to_img,img_to_array,load_img
from keras.applications.xception import preprocess_input
import seaborn as sns
sns.set(color_codes=True)

In [0]:
batch_size=9
num_classes=3
epochs=2
img_width,img_height=256,256

In [110]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
train_data_dir="/content/drive/My Drive/cow/train"
validation_data_dir="/content/drive/My Drive/cow/valid"
test_data_dir="/content/drive/My Drive/cow/test"

In [0]:
train_datagen=ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
    )

In [0]:
  model=applications.xception.Xception(include_top=False,
                                      weights='imagenet',
                                      input_shape=(img_width,img_height,3))

In [0]:
x=model.output
x=GlobalAveragePooling2D()(x)
x=BatchNormalization()(x)
x=Dense(1024,activation='relu')(x)
x=BatchNormalization()(x)
x=Dropout(0.5)(x)
y_pred=Dense(3,activation='softmax')(x)

In [127]:
model_final=Model(inputs=model.input,outputs=y_pred)
model_final.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 127, 127, 32) 864         input_5[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 127, 127, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 127, 127, 32) 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [0]:
for layers in model.layers[:126]:
  layers.trainable=False
for layer in model.layers[126:]:
  layer.trainable=True

In [0]:
train_datagen=ImageDataGenerator(
              rescale=1./255,
              shear_range=0.2,
              zoom_range=0.2,
              horizontal_flip=True)
test_datagen=ImageDataGenerator(rescale=1./255)

In [130]:
train_generator=train_datagen.flow_from_directory(train_data_dir,
                                                 target_size=(img_width,img_height),
                                                 batch_size=batch_size,
                                                 class_mode='categorical')
validation_generator=test_datagen.flow_from_directory(validation_data_dir,
                                                 target_size=(img_width,img_height),
                                                 batch_size=batch_size,
                                                 class_mode='categorical')
test_generator=test_datagen.flow_from_directory(test_data_dir,
                                                 target_size=(img_width,img_height),
                                                 batch_size=batch_size,
                                                 class_mode='categorical')

Found 261 images belonging to 3 classes.
Found 24 images belonging to 3 classes.
Found 22 images belonging to 3 classes.


In [0]:
model_final.compile(loss='categorical_crossentropy',
                   optimizer='nadam',
                   metrics=['accuracy'])   

In [132]:
top_model_weight_path='cow_Xception.h5'
checkpoint=ModelCheckpoint(filepath=top_model_weight_path,
                          verbose=1,
                            save_best_only=True,
                          monitor='val_acc')
early=EarlyStopping(monitor='val_acc',min_delta=0,patience=5,verbose=1,mode='auto')

history=model_final.fit_generator(
              train_generator,
                steps_per_epoch=261//batch_size,
              epochs=epochs,
                validation_data=validation_generator,
                validation_steps=24//batch_size,
                callbacks=[checkpoint,early])

Epoch 1/2
29/29 [==============================] - 153s 5s/step - loss: 1.6823 - acc: 0.7203 - val_loss: 0.7185 - val_acc: 0.8889

Epoch 00001: val_acc improved from -inf to 0.88889, saving model to cow_Xception.h5
Epoch 2/2
29/29 [==============================] - 142s 5s/step - loss: 0.9267 - acc: 0.8276 - val_loss: 0.0272 - val_acc: 1.0000

Epoch 00002: val_acc improved from 0.88889 to 1.00000, saving model to cow_Xception.h5


In [0]:
scores,acc=model_final.evaluate_generator(test_generator,steps=12)

In [0]:
print (acc*100)

In [140]:
img=image.load_img("/content/drive/My Drive/cow/images.jpeg")
target_size=(256,256)
if img.size!=target_size:
  img=img.resize(target_size)
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)
x=preprocess_input(x)
preds=model_final.predict(x)
print("probabilities for sindhi,ongole,kangrej:",preds[0])

probabilities for sindhi,ongole,kangrej: [4.5663136e-17 9.6131232e-17 1.0000000e+00]
